<a href="https://colab.research.google.com/github/mcortes-lopez/CD19_splicing_mutagenesis/blob/main/DeepRiPe_VariantScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up

In [ ]:
pip install concise

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 317kB 15.3MB/s 
  Created wheel for gtfparse: filename=gtfparse-1.2.0-cp36-none-any.whl size=11883 sha256=0eba7df807ac032880a073a355368661e0077d9d81569e506cb4306a89b21c53
  Stored in directory: /root/.cache/pip/wheels/c4/27/96/6ba6fe28cbb162c326823553e3e45ac502160d1340566360f8
Successfully built gtfparse
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
!git clone https://github.com/ohlerlab/DeepRiPe.git

Cloning into 'DeepRiPe'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 795 (delta 1), reused 12 (delta 1), pack-reused 777
Receiving objects: 100% (795/795), 476.59 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Checking out files: 100% (602/602), done.


In [ ]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 512kB 36.6MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8325944c8df5482eb96a3b69a63098eec961a159104e502e2aeae66a3420a2ce
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: 

In [ ]:
!apt-get install bedtools

!pip install pybedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 577 kB of archives.
After this operation, 2,040 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 bedtools amd64 2.26.0+dfsg-5 [577 kB]
Fetched 577 kB in 2s (284 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../bedtools_2.26.0+dfsg-5_amd64.deb ...
Unpacking bedtools (2.26.0+dfsg-5) ...
Setting up bedtools (2.26.0+dfsg-5) ...
     |████████████████████████████████| 12.5MB 334kB/s 
     |████████████████████████████████| 10.0MB 21.9MB/s 
  Created wheel for pybedtools: filenam

In [ ]:
import os
import numpy as np
import h5py
import scipy.io
np.random.seed(7) # for reproducibility

import keras.backend as K
from keras.models import Model, load_model
import numpy as np
import pandas as pd 
import pybedtools

import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append("DeepRiPe/Scripts/")
from IntegratedGradients import *
from util_funcs import *
from plotseqlogo import seqlogo_fig_cross

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
2020-07-01 19:02:21,141 [INFO] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2020-07-01 19:02:21,170 [INFO] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


2020-07-01 19:02:21,717 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

DeepRiPe/Scripts/plotseqlogo.py:44: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  min_coords = np.vstack(data.min(0) for data in polygons_data).min(0)
DeepRiPe/Scripts/plotseqlogo.py:45: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  max_coords = np.vstack(data.max(0) for data in polygons_data).max(0)


In [ ]:
def precision(y_true, y_pred):
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	#TPs=K.sum(K.round(K.clip(y_true * y_pred , 0, 1)))
	predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
	precision = true_positives / (predicted_positives + K.epsilon())
	return precision

def recall(y_true, y_pred):
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	#TPs=K.sum(K.round(K.clip(y_ture * y_pred , 0, 1)))
	possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
	recall = true_positives / (possible_positives + K.epsilon())
	return recall



RBPnames_low=np.array(['MBNL1', 'P53_NONO', 'PUM2', 'QKI', 'AGO3', 'FUS', 'TAF15', 'ZFP36', 'DICER1', 'EIF3A', 'EIF3D', 'EIF3G', 'SSB', 'PAPD5', 'CPSF4', 'CPSF3', 'RTCB', 'FXR1', 'NOP58', 'NOP56', 'FBL', 'LIN28A', 'LIN28B', 'UPF1', 'G35', 'G45', 'XPO5'])
RBPnames_med=np.array(['TARDBP', 'ELAVL2', 'ELAVL3', 'ELAVL4', 'RBM20', 'IGF2BP1', 'IGF2BP2', 'IGF2BP3', 'EWSR1', 'HNRNPD', 'RBPMS', 'SRRM4', 'AGO2', 'NUDT21', 'FIP1L1', 'CAPRIN1', 'FMR1iso7', 'FXR2', 'AGO1', 'L1RE1', 'ORF1'])
RBPnames_high=np.array(['DND1', 'CPSF7', 'CPSF6', 'CPSF1', 'CSTF2', 'CSTF2T', 'ZC3H7B', 'FMR1iso1', 'RBM10', 'MOV10', 'ELAVL1'])


path_to_model = "DeepRiPe/Results/PARCLIP_models/"
path_to_modelhigh= path_to_model + "model_RBPshigh.h5"
path_to_modelmed= path_to_model + "model_RBPsmed.h5"
path_to_modellow= path_to_model + "model_RBPslow.h5"


model_high=load_model(path_to_modelhigh, custom_objects={'precision': precision,'recall': recall }) 
igres_high = integrated_gradients(model_high) 

model_med=load_model(path_to_modelmed, custom_objects={'precision': precision,'recall': recall }) 
igres_med = integrated_gradients(model_med) 

model_low=load_model(path_to_modellow, custom_objects={'precision': precision,'recall': recall }) 
igres_low = integrated_gradients(model_low)

2020-07-01 19:02:21,786 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-07-01 19:02:21,797 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2020-07-01 19:02:21,836 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



2020-07-01 19:02:21,844 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-07-01 19:02:21,847 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-07-01 19:02:22,246 [WARNING] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-07-01 19:02:22,578 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



2020-07-01 19:02:22,844 [WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


In [ ]:
def precision(y_true, y_pred):
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	#TPs=K.sum(K.round(K.clip(y_true * y_pred , 0, 1)))
	predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
	precision = true_positives / (predicted_positives + K.epsilon())
	return precision

def recall(y_true, y_pred):
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	#TPs=K.sum(K.round(K.clip(y_ture * y_pred , 0, 1)))
	possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
	recall = true_positives / (possible_positives + K.epsilon())
	return recall



RBPnames_low=np.array(['MBNL1', 'P53_NONO', 'PUM2', 'QKI', 'AGO3', 'FUS', 'TAF15', 'ZFP36', 'DICER1', 'EIF3A', 'EIF3D', 'EIF3G', 'SSB', 'PAPD5', 'CPSF4', 'CPSF3', 'RTCB', 'FXR1', 'NOP58', 'NOP56', 'FBL', 'LIN28A', 'LIN28B', 'UPF1', 'G35', 'G45', 'XPO5'])
RBPnames_med=np.array(['TARDBP', 'ELAVL2', 'ELAVL3', 'ELAVL4', 'RBM20', 'IGF2BP1', 'IGF2BP2', 'IGF2BP3', 'EWSR1', 'HNRNPD', 'RBPMS', 'SRRM4', 'AGO2', 'NUDT21', 'FIP1L1', 'CAPRIN1', 'FMR1iso7', 'FXR2', 'AGO1', 'L1RE1', 'ORF1'])
RBPnames_high=np.array(['DND1', 'CPSF7', 'CPSF6', 'CPSF1', 'CSTF2', 'CSTF2T', 'ZC3H7B', 'FMR1iso1', 'RBM10', 'MOV10', 'ELAVL1'])


path_to_model = "DeepRiPe/Results/PARCLIP_models/"
path_to_modelhigh= path_to_model + "model_RBPshigh.h5"
path_to_modelmed= path_to_model + "model_RBPsmed.h5"
path_to_modellow= path_to_model + "model_RBPslow.h5"


model_high=load_model(path_to_modelhigh, custom_objects={'precision': precision,'recall': recall }) 
igres_high = integrated_gradients(model_high) 

model_med=load_model(path_to_modelmed, custom_objects={'precision': precision,'recall': recall }) 
igres_med = integrated_gradients(model_med) 

model_low=load_model(path_to_modellow, custom_objects={'precision': precision,'recall': recall }) 
igres_low = integrated_gradients(model_low)


Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


## ENCODE models

In [ ]:


K562_path_to_model = "DeepRiPe/Results/Encode_models/K562/"
K562_path_to_modellow= K562_path_to_model + "eclip_model_encodeK562_low.h5"
K562_path_to_modelmid1= K562_path_to_model + "eclip_model_encodeK562_mid1.h5"
K562_path_to_modelmid2= K562_path_to_model + "eclip_model_encodeK562_mid2.h5"
K562_path_to_modelhigh1= K562_path_to_model + "eclip_model_encodeK562_high1.h5"
K562_path_to_modelhigh2= K562_path_to_model + "eclip_model_encodeK562_high2.h5"

K562_RBPnames_low=np.array(['MTPAP', 'RBM22', 'DHX30', 'DDX6', 'DDX55', 'TRA2A', 'XRN2', 'U2AF1', 'LSM11', 'ZC3H11A', 'NOLC1', 'KHDRBS1', 'GPKOW', 'DGCR8', 'AKAP1', 'FXR1', 'DDX52', 'AATF'])
K562_RBPnames_mid1=np.array(['U2AF2', 'AKAP8L', 'METAP2', 'SMNDC1', 'GEMIN5', 'HNRNPK', 'SLTM', 'SRSF1', 'FMR1', 'SAFB2', 'DROSHA', 'RPS3', 'IGF2BP2', 'ILF3', 'RBFOX2', 'QKI', 'PCBP1', 'ZNF800', 'PUM1'])
K562_RBPnames_mid2=np.array(['EFTUD2', 'LIN28B', 'AGGF1', 'HNRNPL', 'SND1', 'GTF2F1', 'EIF4G2', 'TIA1', 'TARDBP', 'FXR2', 'HNRNPM', 'IGF2BP1', 'PUM2', 'FAM120A', 'DDX3X', 'MATR3', 'FUS', 'GRWD1', 'PABPC4'])
K562_RBPnames_high1=np.array(['BUD13', 'PTBP1', 'DDX24', 'EWSR1', 'RBM15'])
K562_RBPnames_high2=np.array(['SF3B4', 'YBX3', 'UCHL5', 'KHSRP', 'ZNF622', 'NONO', 'EXOSC5', 'PRPF8', 'CSTF2T', 'AQR', 'UPF1'])

K562_RBP_low=load_model(K562_path_to_modellow, custom_objects={'precision': precision,'recall': recall }) 
K562_RBP_mid1=load_model(K562_path_to_modelmid1, custom_objects={'precision': precision,'recall': recall })
K562_RBP_mid2=load_model(K562_path_to_modelmid2, custom_objects={'precision': precision,'recall': recall }) 
K562_RBP_high1=load_model(K562_path_to_modelhigh1, custom_objects={'precision': precision,'recall': recall }) 
K562_RBP_high2=load_model(K562_path_to_modelhigh2, custom_objects={'precision': precision,'recall': recall }) 

K562_igres_low = integrated_gradients(K562_RBP_low)
K562_igres_mid1 = integrated_gradients(K562_RBP_mid1)
K562_igres_mid2 = integrated_gradients(K562_RBP_mid2)
K562_igres_high1 = integrated_gradients(K562_RBP_high1)
K562_igres_high2 = integrated_gradients(K562_RBP_high2)


HepG2_path_to_model = "DeepRiPe/Results/Encode_models/HepG2/"
HepG2_path_to_modellow= HepG2_path_to_model + "eclip_model_encodeHepG2_low.h5"
HepG2_path_to_modelmid1= HepG2_path_to_model + "eclip_model_encodeHepG2_mid1.h5"
HepG2_path_to_modelmid2= HepG2_path_to_model + "eclip_model_encodeHepG2_mid2.h5"
HepG2_path_to_modelhigh1= HepG2_path_to_model + "eclip_model_encodeHepG2_high1.h5"
HepG2_path_to_modelhigh2= HepG2_path_to_model + "eclip_model_encodeHepG2_high2.h5"


HepG2_RBPnames_low=np.array(['RBM22', 'GRSF1', 'CDC40', 'NOLC1', 'FKBP4', 'DGCR8', 'ZC3H11A', 'XRN2', 'SLTM', 'DDX55', 'TIA1', 'SRSF1', 'U2AF1', 'RBM15'])
HepG2_RBPnames_mid1=np.array(['LSM11', 'NKRF', 'SUB1', 'NCBP2', 'UCHL5', 'LIN28B', 'IGF2BP3', 'SF3A3', 'AGGF1', 'DROSHA', 'DDX59', 'CSTF2', 'DKC1', 'EIF3H', 'FUBP3', 'SFPQ', 'HNRNPC', 'ILF3', 'TIAL1', 'HLTF', 'ZNF800', 'PABPN1', 'YBX3', 'FXR2'])
HepG2_RBPnames_mid2=np.array(['GTF2F1', 'IGF2BP1', 'HNRNPK', 'XPO5', 'RPS3', 'SF3B4', 'LARP4', 'BUD13', 'SND1', 'G3BP1', 'AKAP1', 'KHSRP'])
HepG2_RBPnames_high1=np.array(['DDX3X', 'PCBP2', 'FAM120A', 'HNRNPL', 'RBFOX2', 'PTBP1', 'MATR3', 'EFTUD2', 'PRPF4', 'UPF1'])
HepG2_RBPnames_high2= np.array(['GRWD1', 'PRPF8', 'PPIG', 'CSTF2T', 'QKI', 'U2AF2', 'SUGP2', 'HNRNPM', 'AQR', 'BCLAF1'])
  
HepG2_RBP_low=load_model(HepG2_path_to_modellow, custom_objects={'precision': precision,'recall': recall }) 
HepG2_RBP_mid1=load_model(HepG2_path_to_modelmid1, custom_objects={'precision': precision,'recall': recall })
HepG2_RBP_mid2=load_model(HepG2_path_to_modelmid2, custom_objects={'precision': precision,'recall': recall }) 
HepG2_RBP_high1=load_model(HepG2_path_to_modelhigh1, custom_objects={'precision': precision,'recall': recall }) 
HepG2_RBP_high2=load_model(HepG2_path_to_modelhigh2, custom_objects={'precision': precision,'recall': recall }) 

HepG2_igres_low = integrated_gradients(HepG2_RBP_low)
HepG2_igres_mid1 = integrated_gradients(HepG2_RBP_mid1)
HepG2_igres_mid2 = integrated_gradients(HepG2_RBP_mid2)
HepG2_igres_high1 = integrated_gradients(HepG2_RBP_high1)
HepG2_igres_high2 = integrated_gradients(HepG2_RBP_high2)


Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.


In [ ]:
##Function to prepare the input for the model from variants bedfile
##The bedfile should be as follows : chr	start	end	  variant_name    W/M     strand  where(start=end)

def encode_variant_bedline(bedline,genomefasta,flank_size=75):
    mut_a = bedline[4].split("/")[1]
    strand = bedline[5]
    if len(mut_a)==1:
        wild = pybedtools.BedTool(bedline[0] + "\t" + str(int(bedline[1])-flank_size) + "\t"  + str(int(bedline[2])+flank_size) + "\t" + 
                                  bedline[3] + "\t" + str(mut_a) + "\t" + bedline[5], from_string=True )
        if strand == "-" :
            mut_pos= flank_size
        else:
            mut_pos= flank_size-1
                            
        #wild = pybedtools.BedTool(bedline[0] + "\t" + bedline[1] + "\t" + bedline[2] + "\t" + bedline[3] + "\t"+ bedline[4] + "\t" + bedline[5], from_string=True)
        wild = wild.sequence(fi=genomefasta, tab=True, s=True)
        fastalist = open(wild.seqfn).read().split("\n")
        del fastalist[-1]
        seqs=[fasta.split("\t")[1] for fasta in fastalist]
        mut=seqs[0]
        mut = list(mut)
        mut[mut_pos] = mut_a
        mut = "".join(mut)
        seqs.append(mut)
        encoded_seqs =np.array([seq_to_1hot(seq) for seq in seqs])
        encoded_seqs = np.transpose(encoded_seqs,axes=(0,2,1))
        return(encoded_seqs)
                   
    


##Function to score variants using models with both sequence and region 
def score_variant_withregion(model,RBP_index,variant_bed,genomefasta,tr=0.1):
    reg_coded = np.full((250, 4), 0.25)
    region=np.array([reg_coded,reg_coded])
    score_list = []
    bed_list = []
    seqs_list = []
    for bedline in variant_bed:
        encoded_seqs = encode_variant_bedline(bedline,genomefasta)
        if encoded_seqs is not None:
            pred = model.predict([encoded_seqs,region])[:,RBP_index]
            score = (pred[1]-pred[0])
            if abs(score) > tr:
                score_list.append(score)
                bed_list.append(bedline)
                seqs_list.append(encoded_seqs)
    return(score_list,bed_list,seqs_list)


####Function to score variants using models with only sequence
def score_variant_onlyseq(model,RBP_index,variant_bed,genomefasta,tr=0.1):
    score_list = []
    bed_list = []
    seqs_list = []
    for bedline in variant_bed:
        encoded_seqs = encode_variant_bedline(bedline,genomefasta)
        if encoded_seqs is not None:
            pred = model.predict(encoded_seqs)[:,RBP_index]
            score = (pred[1]-pred[0])
            if abs(score) > tr:
                score_list.append(score)
                bed_list.append(bedline)
                seqs_list.append(encoded_seqs)
    return(score_list,bed_list,seqs_list) 


####Function to plot attribution map for variant using models with both sequence and region 
def plot_variant_map_withregion(bedlines,igres,RBP_index,genomefasta,figsave_path=""):
    for bedline in bedlines[0:bedlines.count()]:
        encoded_seqs = encode_variant_bedline(bedline,genomefasta)
        reg_coded = np.full((250, 4), 0.25)
        ex_seq = np.array([igres.explain([encoded_seqs[i],reg_coded],outc=RBP_index,reference=False)[0] for i in [0,1]])
        plt.close("all")
        seqlogo_fig_cross(np.transpose(ex_seq[:,50:100,:4],axes=(1,2,0)), vocab="RNA", figsize=(4,1.5), ncol=1, crosssite=True, cross_positions=[(24.5,25.5),(24.5,25.5),(24.5,25.5)])
        #plt.savefig(os.path.join(figsave_path,bedline[3]+"_"+str(RBP_index)+"_region.pdf"),dpi=300)
        #plt.savefig(os.path.join(figsave_path,bedline[3]+"_"+str(RBP_index)+"_region.png"),dpi=300)
        plt.show()
        
####Function to plot attribution map for variant using models with only sequence
def plot_variant_map_onlyseq(bedlines,igres,RBP_index,genomefasta,figsave_path=""):
    for bedline in bedlines[0:bedlines.count()]:
        encoded_seqs = encode_variant_bedline(bedline,genomefasta)
        ex_seq = np.array([igres.explain(encoded_seqs[i],outc=RBP_index,reference=False)[0] for i in [0,1]])
        plt.close("all")
        seqlogo_fig_cross(np.transpose(ex_seq[:,50:100,:4],axes=(1,2,0)), vocab="RNA", figsize=(4,1.5), ncol=1, crosssite=True, cross_positions=[(24.5,25.5),(24.5,25.5),(24.5,25.5)])
        #plt.savefig(os.path.join(figsave_path,bedline[3]+"_"+str(RBP_index)+"_region.pdf"),dpi=300)
        #plt.savefig(os.path.join(figsave_path,bedline[3]+"_"+str(RBP_index)+"_region.png"),dpi=300)
        plt.show()

Testing the models with CD19 variants

In [ ]:
#genomefa="drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/CD19_WT.minigene.fa"

Getting the hg38 genome

In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

--2020-07-01 19:05:25--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  21.5MB/s    in 62s     

2020-07-01 19:06:28 (15.2 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]



In [ ]:
!gunzip hg38.fa.gz

In [ ]:
genomefa="hg38.fa"

Input some variants

### Variants from SNP153 in the minigene region 

It does not seem to work with the minigene sequence as genome only. Therefore I am trying with the hg38 reference and testing some SNPs in the minigene region.

In [ ]:
variant_bed=pybedtools.BedTool('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/all_potential_snps.bed')

In [ ]:
variant_bed.head()

chr16	28931871	28931871	G1A	G/A	+
 chr16	28931871	28931871	G1C	G/C	+
 chr16	28931871	28931871	G1T	G/T	+
 chr16	28931872	28931872	A2C	A/C	+
 chr16	28931872	28931872	A2G	A/G	+
 chr16	28931872	28931872	A2T	A/T	+
 chr16	28931873	28931873	C3A	C/A	+
 chr16	28931873	28931873	C3G	C/G	+
 chr16	28931873	28931873	C3T	C/T	+
 chr16	28931874	28931874	C4A	C/A	+
 

### Looping variants across all the models

In [ ]:
def alt_variant_binding(rbpmodel,rbpnames):
  model_result = defaultdict(list)   
  for rbp in rbpnames:
      model = rbpmodel
      RBP_index = np.where(rbpnames == rbp)[0][0]
      rbp_score_list, rbp_bed_list, rbp_seqs_list = score_variant_withregion(model,RBP_index,variant_bed,genomefa,tr=0.1)
      if len(rbp_score_list) != 0:
        model_result[rbp].append(rbp_score_list)
        model_result[rbp].append(rbp_bed_list)
        model_result[rbp].append(rbp_seqs_list) 
  return(model_result)


PAR CLIP models

In [ ]:
parclip_low_res = alt_variant_binding(model_low, RBPnames_low)
parclip_mid_res = alt_variant_binding(model_med, RBPnames_med)
parclip_high_res = alt_variant_binding(model_high, RBPnames_high)


eCLIP models

In [ ]:
##Function to score variants using models with both sequence and region 
def score_variant_withregion(model,RBP_index,variant_bed,genomefasta,tr=0.1):
    reg_coded = np.full((250, 4), 0.25)
    region=np.array([reg_coded,reg_coded])
    score_list = []
    bed_list = []
    seqs_list = []
    for bedline in variant_bed:
        encoded_seqs = encode_variant_bedline(bedline,genomefasta,  flank_size=100)
        if encoded_seqs is not None:
            pred = model.predict([encoded_seqs,region])[:,RBP_index]
            score = (pred[1]-pred[0])
            if abs(score) > tr:
                score_list.append(score)
                bed_list.append(bedline)
                seqs_list.append(encoded_seqs)
    return(score_list,bed_list,seqs_list)


In [ ]:
eclip_HepG2_low_res = alt_variant_binding(HepG2_RBP_low, HepG2_RBPnames_low)
eclip_HepG2_mid1_res = alt_variant_binding(HepG2_RBP_mid1, HepG2_RBPnames_mid1)
eclip_HepG2_mid2_res = alt_variant_binding(HepG2_RBP_mid2, HepG2_RBPnames_mid2)
eclip_HepG2_high1_res = alt_variant_binding(HepG2_RBP_high1, HepG2_RBPnames_high1)
eclip_HepG2_high2_res = alt_variant_binding(HepG2_RBP_high2, HepG2_RBPnames_high2)

index file hg38.fa.fai not found, generating...


In [ ]:
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/HepG2_eclip_low_res.npy', eclip_HepG2_low_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/HepG2_eclip_med1_res.npy', eclip_HepG2_mid1_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/HepG2_eclip_med2_res.npy', eclip_HepG2_mid2_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/HepG2_eclip_high1_res.npy', eclip_HepG2_high1_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/HepG2_eclip_high2_res.npy', eclip_HepG2_high2_res) 



In [ ]:
eclip_K562_low_res = alt_variant_binding(K562_RBP_low, K562_RBPnames_low)
eclip_K562_mid1_res = alt_variant_binding(K562_RBP_mid1, K562_RBPnames_mid1)
eclip_K562_mid2_res = alt_variant_binding(K562_RBP_mid2, K562_RBPnames_mid2)
eclip_K562_high1_res = alt_variant_binding(K562_RBP_high1, K562_RBPnames_high1)
eclip_K562_high2_res = alt_variant_binding(K562_RBP_high2, K562_RBPnames_high2)

In [ ]:
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/K562_eclip_low_res.npy', eclip_K562_low_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/K562_eclip_med1_res.npy', eclip_K562_mid1_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/K562_eclip_med2_res.npy', eclip_K562_mid2_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/K562_eclip_high1_res.npy', eclip_K562_high1_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/K562_eclip_high2_res.npy', eclip_K562_high2_res) 

In [ ]:
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/parclip_low_res.npy', parclip_low_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/parclip_med_res.npy', parclip_mid_res) 
np.save('drive/My Drive/CD19_project/RBP_regulators/DeepRiPe_files/parclip_high_res.npy', parclip_high_res) 
